In [ ]:
import pandas as pd
import json

file_path = 'terrorism_dataset.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)

attack_map = {
    'Bombing/Explosion': 'explosion',
    'Armed Assault': 'armed_assault',
    'Assassination': 'assassination',
    'Hostage Taking (Kidnapping)': 'hostage_taking',
    'Hostage Taking (Barricade Incident)': 'hostage_taking',
    'Facility/Infrastructure Attack': 'infrastructure_attack',
    'Hijacking': 'others',
    'Unknown': 'others',
    'Unarmed Assault': 'others'
}

df['normalized_type'] = df['attacktype1_txt'].map(attack_map).fillna('others')

country_first_appearance = {}
for _, row in df.iterrows():
    year = str(row['iyear'])
    country = row['country_txt']
    key = (year, country)
    if key not in country_first_appearance:
        country_first_appearance[key] = len([k for k in country_first_appearance if k[0] == year])

grouped = df.groupby(['iyear', 'country_txt', 'normalized_type']).size().reset_index(name='count')
idx = grouped.groupby(['iyear', 'country_txt'])['count'].idxmax()
top_attacks = grouped.loc[idx]

results = []
for _, row in top_attacks.iterrows():
    year = str(row['iyear'])
    country = row['country_txt']
    results.append({
        'year': year,
        'country': country,
        'type': row['normalized_type'],
        'count': int(row['count']),
        'order': country_first_appearance.get((year, country), 999)
    })

results.sort(key=lambda x: (int(x['year']), x['order']))

output_dict = {}
for r in results:
    year = r['year']
    if year not in output_dict:
        output_dict[year] = {}
    output_dict[year][r['country']] = {
        "type": r['type'],
        "count": r['count']
    }

output_filename = 'globe_attacks_counts1.json'
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(output_dict, f, indent=1)

print(f"File {output_filename} generated.")

File globe_attacks_counts1.json generato correttamente.
